# Nuts Region SQL Generator 
### Last Modified: February 24, 2021

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.

In [ ]:
# Widgets
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from ipywidgets import Layout

# Imports
import pandas as pd
import numpy as np

from datetime import timedelta, datetime
import calendar

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

generator_sheet_url = "https://docs.google.com/spreadsheets/d/1nlkb3BlEuu3oW2eHnMZTptDHYz16HtUtnn_oAtwh_YU"

# Print Generator Sheet URL
print("Go to {0} to view SQL Generator Sheet".format(generator_sheet_url))

In [ ]:
# Allows window height to be larger
# Doesn't seem to work in Binder?
HTML('''
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>''');

In [ ]:
# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Loads Nuts Region data

In [ ]:
# Initializes nuts dictionary
nuts = {}

for nuts_region in ["Nuts1", "Nuts2", "Nuts3"] :
    # Loads nuts region sheet
    nuts_region_sheet = gs.open_by_url(generator_sheet_url).worksheet(nuts_region).get_all_values()
    nuts[nuts_region] = pd.DataFrame(nuts_region_sheet[1:], columns = nuts_region_sheet[0])
    print("Loaded {0} Region data".format(nuts_region))

# Chooses the Nuts Region and other parameters

In [ ]:
# Defines distance calculation function
def distance_funct(start_lat, start_lng, lat, lng) :
    # Not sure why it is multiplied by 6371
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

# Dictionary of category names to numbers
categories = {"Furniture" : 1,
              "Removals" : 2, 
              "Cars" : 4, 
              "Pianos" : 20,
              "Bikes" : 27, 
              "Hourly" : 73}

In [ ]:
# Nuts options
nuts_dropdown = {"Nuts3" : widgets.Dropdown(
                            options = np.insert(np.array(nuts["Nuts3"]["name"]), 0, "Nuts3 Region"),
                            value = "Nuts3 Region",
                            description = "Nuts3:",
                            style = {"description_width":"initial"}),
                 "Nuts2" : widgets.Dropdown(
                            options = np.insert(np.array(nuts["Nuts2"]["name"]), 0, "Nuts2 Region"),
                            value = "Nuts2 Region",
                            description = "Nuts2:",
                            style = {"description_width":"initial"}),
                 "Nuts1" : widgets.Dropdown(
                            options = np.insert(np.array(nuts["Nuts1"]["name"]), 0, "Nuts1 Region"),
                            value = "Nuts1 Region",
                            description = "Nuts1:",
                            style = {"description_width":"initial"})
                }
nuts_button = {"Nuts3" : widgets.Button(description = "Choose Nuts3", layout = Layout(width = "120px")),
               "Nuts2" : widgets.Button(description = "Choose Nuts2", layout = Layout(width = "120px")),
               "Nuts1" : widgets.Button(description = "Choose Nuts1", layout = Layout(width = "120px"))
              }
nuts_output = widgets.Output()

# Exclusion range options
to_from_radio = widgets.RadioButtons(
    options = ["To", "From", "To and From"],
    value = None,
    description = "To or From Nuts Region:",
    disabled = False,
    style = {"description_width":"initial"})
exclusion_distance_box = widgets.FloatText(
    value = 200.0,
    description = "Exclusion Distance (km):",
    disabled = False,
    style = {"description_width":"initial"})
category_dropdown = widgets.Dropdown(
    options = categories.keys(),
    value = "Furniture",
    description = "Job Category:",
    style = {"description_width":"initial"})
percent_box = widgets.FloatText(
    value = 1.0,
    description = "Percent Value:",
    disabled = False,
    style = {"description_width":"initial"})
lifetime_box = widgets.FloatText(
    value = 1.0,
    description = "Lifetime Value:",
    disabled = False,
    style = {"description_width":"initial"})
start_date = widgets.DatePicker(
    description = "Start Date:",
    disabled = False)
end_date = widgets.DatePicker(
    description = "End Date:",
    disabled = False)
day_of_week_exclusion_select = widgets.SelectMultiple(
    options = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    value = [],
    description = "Days of Week to Exclude:",
    disabled = False,
    style = {"description_width":"initial"})
day_of_week_exclusion_button = widgets.Button(description = "Clear Days of Week Selection", layout = Layout(width = "200px"))
day_of_week_exclusion_button_weekend = widgets.Button(description = "Select Weekend", layout = Layout(width = "140px"))
day_of_week_exclusion_button_sunday = widgets.Button(description = "Select Sunday", layout = Layout(width = "140px"))
generate_button = widgets.Button(description = "Generator SQL Query", layout = Layout(width = "160px"))
reset_exclusion_options_button = widgets.Button(description = "Reset Exclusion Options", layout = Layout(width = "180px"))
exclusion_output = widgets.Output()

# Initializes data
data = None

# Specifi functions for nuts region
def nuts3(_) :
    nuts_generic("Nuts3")
def nuts2(_) :
    nuts_generic("Nuts2")
def nuts1(_) :
    nuts_generic("Nuts1")
    
# Generic function for nuts region selection
def nuts_generic(nuts_region) :
    # Allows data to be saved outside the function
    global data
    
    # Clears exclusion_output
    with exclusion_output :
        clear_output()
    
    with nuts_output :
        # Clears output for overall nuts_output
        clear_output()
        
        # Resets options for other nuts region dropdowns
        for nuts_dropdown_option in nuts_dropdown.keys() :
            if nuts_dropdown_option != nuts_region :
                nuts_dropdown[nuts_dropdown_option].value = "{0} Region".format(nuts_dropdown_option)
        
        # Ensures standard options are not selected
        if nuts_dropdown[nuts_region].value != "{0} Region".format(nuts_region) :
            # Finds data for this nuts region
            data = nuts[nuts_region].loc[nuts[nuts_region]["name"] == nuts_dropdown[nuts_region].value].iloc[0]
            data["nuts_region"] = nuts_region
            print("Selected {0} ({1}) from {2}".format(data["name"], data["code"], nuts_region))
            
            # Displays exclusion widgets
            display(widgets.VBox([widgets.HBox([to_from_radio, exclusion_distance_box]), 
                                  widgets.HBox([category_dropdown, percent_box, lifetime_box]),
                                  widgets.HBox([start_date, end_date]),
                                  widgets.HBox([day_of_week_exclusion_select, day_of_week_exclusion_button, day_of_week_exclusion_button_weekend, day_of_week_exclusion_button_sunday]),
                                  widgets.HBox([generate_button, reset_exclusion_options_button]),
                                  exclusion_output]))
        
# Excludes nuts regions near the selected nuts region
def exclusion_funct(_) :
    # Allows for data to be saved globally
    global data
    
    with exclusion_output :
        # Clears output
        clear_output()
        
        # Ensures exclusion options are selected
        if to_from_radio.value != None and exclusion_distance_box.value > 0 and percent_box.value >= 0 and lifetime_box.value >= 0 and end_date.value != None and start_date.value != None and (end_date.value - start_date.value).days >= 0 :
            # Saves exclusion options
            data["to_from"] = to_from_radio.value
            data["exclusion_distance"] = exclusion_distance_box.value
            data["category"] = category_dropdown.value
            data["percent"] = percent_box.value
            data["lifetime"] = lifetime_box.value
            data["start_date"] = start_date.value
            data["end_date"] = end_date.value
            data["weekday_exclude"] = day_of_week_exclusion_select.value
            print("Selected {0} {1} ({2}) for other {3} Regions within {4} km".format(data["to_from"], data["name"], data["code"], data["nuts_region"], data["exclusion_distance"]))
            print("Setting {0} jobs with a {1} percent value and {2} lifetime value between {3} and {4}{5}".format(data["category"], data["percent"], data["lifetime"], data["start_date"], data["end_date"], " excluding {0}".format(", ".join(data["weekday_exclude"])) if len(data["weekday_exclude"]) > 0 else ""))

            nuts_other = nuts[data["nuts_region"]]
            nuts_other["distance"] = [distance_funct(float(data["average_lat"]), float(data["average_lng"]), float(lat), float(lng)) for lat,lng in zip(nuts_other["average_lat"], nuts_other["average_lng"])]
            nuts_other = nuts_other.loc[nuts_other["distance"] <= data["exclusion_distance"]]
            print("\n{0} {1} Regions within {2} km to {3} ({4})".format(len(nuts_other), data["nuts_region"], data["exclusion_distance"], data["name"], data["code"]))
            
            exclusion_script = prepare_exclusion_script(data, nuts_other)
            save_exclusion_script(data, exclusion_script)
                
def day_of_week_exclusion_reset(_) :
    # Resets day_of_week_exclusion_select
    day_of_week_exclusion_select.value = []
    
def day_of_week_exclusion_weekend(_) :
    # Resets day_of_week_exclusion_select
    day_of_week_exclusion_select.value = ["Saturday", "Sunday"]
    
def day_of_week_exclusion_sunday(_) :
    # Resets day_of_week_exclusion_select
    day_of_week_exclusion_select.value = ["Sunday"]
    
def reset_exclusion_options(_) :
    # Resets exclusion options
    to_from_radio.value = None
    exclusion_distance_box.value = 200
    category_dropdown.value = "Furniture"
    percent_box.value = 1.0
    lifetime_box.value = 1.0
    start_date.value = None
    end_date.value = None
    day_of_week_exclusion_select.value = []

# Assigns functions to buttons
nuts_button["Nuts1"].on_click(nuts1)
nuts_button["Nuts2"].on_click(nuts2)
nuts_button["Nuts3"].on_click(nuts3)
generate_button.on_click(exclusion_funct)
day_of_week_exclusion_button.on_click(day_of_week_exclusion_reset)
day_of_week_exclusion_button_weekend.on_click(day_of_week_exclusion_weekend)
day_of_week_exclusion_button_sunday.on_click(day_of_week_exclusion_sunday)
reset_exclusion_options_button.on_click(reset_exclusion_options)

widgets.VBox([widgets.HBox([nuts_dropdown["Nuts1"], nuts_button["Nuts1"]]), 
              widgets.HBox([nuts_dropdown["Nuts2"], nuts_button["Nuts2"]]), 
              widgets.HBox([nuts_dropdown["Nuts3"], nuts_button["Nuts3"]]), 
              nuts_output])

In [ ]:
# Dictionary of nuts types
nuts_type = {"Nuts1" : 21,
             "Nuts2" : 22,
             "Nuts3" : 23}

# Gets an array of strings that is the exclusion script
def prepare_exclusion_script(data, nuts_other) :
    # Initializes the exclusion script as an array to help with the new lines when saving
    exclusion_script = [["INSERT INTO margin (percent,lifetime,start,end,type,category_id,date,status) VALUES"]]
    
    # Gets the string for each date between the start and end date
    dates = []
    for i in range((data["end_date"] - data["start_date"]).days + 1) :
        date = data["start_date"] + timedelta(days = i)
        # Excludes day of week that are in data["weekday_exclude"]
        if calendar.day_name[date.weekday()] not in data["weekday_exclude"] :
            dates.append(date.strftime("%Y-%m-%d"))
    
    # Runs through each date and saves the exclusion script for both To and/or From cases
    if "To" in data["to_from"] :
        for date in dates :
            for _,location in nuts_other.iterrows() :
                exclusion_script.append(["({0},{1},'{2}','{3}',{4},{5},'{6}',1),".format(data["percent"], 
                                                                                        data["lifetime"], 
                                                                                        location["code"], 
                                                                                        data["code"], 
                                                                                        nuts_type[data["nuts_region"]], 
                                                                                        categories[data["category"]], 
                                                                                        date)])
            
    if "From" in data["to_from"] :
        for date in dates :
            for _,location in nuts_other.iterrows() :
                exclusion_script.append(["({0},{1},'{2}','{3}',{4},{5},'{6}',1),".format(data["percent"], 
                                                                                        data["lifetime"], 
                                                                                        data["code"], 
                                                                                        location["code"], 
                                                                                        nuts_type[data["nuts_region"]], 
                                                                                        categories[data["category"]], 
                                                                                        date)])
    
    # Removes last comma from script
    exclusion_script[-1][0] = exclusion_script[-1][0][:-1]
    
    return exclusion_script

def save_exclusion_script(data, exclusion_script) :
    # Opens Output_Script sheet
    output_script_sheet = gs.open_by_url(generator_sheet_url).worksheet("Output_Script")
    output_script_sheet.clear()
    
    # Saves exclusion script to Output_Script sheet
    output_script_sheet.update("A1", exclusion_script)
    print("\nSaved exclusion script to {0}".format(generator_sheet_url))
    
    # Resets information sheet
    gs.open_by_url(generator_sheet_url).values_clear("Information!A2:N2")
    information_sheet = gs.open_by_url(generator_sheet_url).worksheet("Information")
    
    # Saves run data
    information_sheet.update("A2", [[datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "{0} ({1}) [{2}]".format(data["name"],data["code"],data["nuts_region"]), data["exclusion_distance"], data["to_from"], data["category"], data["percent"], data["lifetime"], data["start_date"].strftime("%Y-%m-%d"), data["end_date"].strftime("%Y-%m-%d"), ", ".join(data["weekday_exclude"])]])

# Done